In [5]:
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline
import torch
import os
from PIL import Image
from torchvision import transforms

# Clear GPU cache
torch.cuda.empty_cache()

# Determine the device to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize the ControlNet model and pipeline
controlnet = ControlNetModel.from_pretrained("/home/wisley/train_controlnet/output/checkpoint-10000/controlnet")
pipeline = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet
).to(device=device, dtpyes=torch.float16)
pipeline.enable_xformers_memory_efficient_attention()

# Define the transformation for resizing images
transform_sus = transforms.Compose([
    transforms.Resize((360, 640)),  # Resize the image to 360x640 pixels
])

def load_images(scene_name, path, batch_size):
    """
    Load images from a specified directory, apply transformations, 
    and return batches of images along with their frame names.
    """
    batch = []
    frame_names = []

    # List all image files in the directory
    all_files = os.listdir(os.path.join(path, scene_name))
    filtered_files = [x for x in all_files if x.endswith(".png")]

    batches = []
    frame_name_batches = []

    for i, file in enumerate(filtered_files):
        image = Image.open(os.path.join(path, scene_name, file))
        image = transform_sus(image)  # Apply the transformation

        batch.append(image)
        frame_names.append(file)

        if len(batch) == batch_size or (i == len(filtered_files) - 1 and batch):
            batches.append(batch)
            frame_name_batches.append(frame_names)
            batch = []
            frame_names = []

    return batches, frame_name_batches

# Define paths
path = "/mnt/e/13_Jasper_diffused_samples/nuscene_frames"
output_base_path = "/mnt/d/diffused/clean"

# Filter directories based on the naming pattern
all_dirs = os.listdir(path)
filtered_dirs = [x for x in all_dirs if "_seg" in x]
batch_size = 4
# Process each directory
for dir in filtered_dirs:
    batches, frame_name_batches = load_images(dir, path, batch_size)
    for batch, frame_names in zip(batches, frame_name_batches):
        
        prompt = ["a driving scene"] * len(batch)
        negative_prompt = ["simulation, blurry"] * len(batch)

        # Check if the output files already exist
        skip_batch = all(os.path.exists(os.path.join(output_base_path, dir, frame_name)) for frame_name in frame_names)

        if skip_batch:
            continue
        
        # Generate images using the pipeline
        generated_images = pipeline(prompt, batch,  negative_prompt=negative_prompt).images

        for generated_image, frame_name in zip(generated_images, frame_names):
            output_dir = os.path.join(output_base_path, dir)
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, frame_name)
            generated_image.save(output_path)


Using device: cuda


  0%|          | 0/50 [00:00<?, ?it/s]


KeyboardInterrupt: 

 78%|███████▊  | 39/50 [00:22<00:06,  1.72it/s]


KeyboardInterrupt: 

In [3]:
import os
import torch
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from multiprocessing import Pool

batch_size = 4
transform_sus = transforms.Compose([
    transforms.Resize((360, 640)),  # Resize the image to 360x640 pixels
])

def load_images(scene_name, path, batch_size):
    batch = []
    frame_names = []

    # check how many images are in the directory
    all_files = os.listdir(os.path.join(path, scene_name))
    filtered_files = [x for x in all_files if x.endswith(".png")]
    
    batches = []
    frame_name_batches = []
    for i, file in enumerate(filtered_files):
        image = Image.open(os.path.join(path, scene_name, file))
        
        # apply the transformation
        image = transform_sus(image)

        batch.append(image)
        frame_names.append(file)

        if len(batch) == batch_size or (i == len(filtered_files) - 1 and batch):
            batches.append(batch)
            frame_name_batches.append(frame_names)
            batch = []
            frame_names = []

    return batches, frame_name_batches

def process_batch(args):
    batch, frame_names, dir, path, output_base_path, pipeline = args
    prompt = ["a driving scene" for _ in range(len(batch))]
    negative_prompt = ["simulation, blurry" for _ in range(len(batch))]

    # check if the files already exist 
    skip_batch = True
    for frame_name in frame_names:
        output_path = os.path.join(output_base_path, dir, frame_name)
        if not os.path.exists(output_path):
            skip_batch = False
            break

    if skip_batch:
        return

    generated_images = pipeline(prompt, batch, guidance_scale=9.5, negative_prompt=negative_prompt).images

    for generated_image, frame_name in zip(generated_images, frame_names):
        # check if the directory exists
        output_dir = os.path.join(output_base_path, dir)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        output_path = os.path.join(output_dir, frame_name)
        generated_image.save(output_path)

path = "/mnt/e/13_Jasper_diffused_samples/nuscene_frames"
output_base_path = "/mnt/d/diffused/clean"
all_dirs = os.listdir(path)
filtered_dirs = [x for x in all_dirs if "_seg" in x]


tasks = []
for dir in filtered_dirs:
    batches, frame_name_batches = load_images(dir, path, batch_size)
    for batch, frame_names in zip(batches, frame_name_batches):
        tasks.append((batch, frame_names, dir, path, output_base_path, pipeline))

with Pool(processes=os.cpu_count()) as pool:
    pool.map(process_batch, tasks)

KeyboardInterrupt: 